# Chat Examples

In [70]:
from openai import OpenAI

# Configure client to use local server
client = OpenAI(
    base_url="http://localhost:10240/v1",  # Point to local server
    api_key="not-needed"  # API key is not required for local server
)

## v1/chat/completions

You can directly test using the curl method, as follows:

```shell
curl http://localhost:10240/v1/chat/completions \
  -H "Content-Type: application/json" \
  -d '{
    "model": "mlx-community/Llama-3.2-3B-Instruct-4bit",
    "messages": [
      {
        "role": "system",
        "content": "You are a helpful assistant."
      },
      {
        "role": "user",
        "content": "Hello!"
      }
    ]
  }'

```

You can also use OpenAI's Python SDK in the project for access, which can basically be done without feeling. As follows:

In [71]:
completion = client.chat.completions.create(
  model="mlx-community/Llama-3.2-3B-Instruct-4bit",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello!"}
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content="Hello! It's nice to meet you. Is there something I can help you with or would you like to chat for a bit?", refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)


In [72]:
completion

ChatCompletion(id='chatcmpl-ac6138de1f', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Hello! It's nice to meet you. Is there something I can help you with or would you like to chat for a bit?", refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1732542634, model='mlx-community/Llama-3.2-3B-Instruct-4bit', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=28, prompt_tokens=3, total_tokens=31, completion_tokens_details=None, prompt_tokens_details=None))

In [78]:
response = client.chat.completions.create(
    model="mlx-community/Llama-3.2-3B-Instruct-4bit",
    messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello!"}
    ],
    temperature=0,
    stream=True  # this time, we set stream=True
)

for chunk in response:
    print(chunk)
    print(chunk.choices[0].delta.content)
    print("****************")

ChatCompletionChunk(id='chatcmpl-0870a60d5b', choices=[Choice(delta=ChoiceDelta(content='', function_call=None, refusal=None, role='assistant', tool_calls=None), finish_reason=None, index=0, logprobs=None)], created=1732542953, model='mlx-community/Llama-3.2-3B-Instruct-4bit', object='chat.completion.chunk', service_tier=None, system_fingerprint=None, usage=None)

****************
ChatCompletionChunk(id='chatcmpl-0870a60d5b', choices=[Choice(delta=ChoiceDelta(content='', function_call=None, refusal=None, role='assistant', tool_calls=None), finish_reason=None, index=0, logprobs=None)], created=1732542953, model='mlx-community/Llama-3.2-3B-Instruct-4bit', object='chat.completion.chunk', service_tier=None, system_fingerprint=None, usage=None)

****************
ChatCompletionChunk(id='chatcmpl-0870a60d5b', choices=[Choice(delta=ChoiceDelta(content='Hello!', function_call=None, refusal=None, role='assistant', tool_calls=None), finish_reason=None, index=0, logprobs=None)], created=1732542953

## logprobs

[openai cookbook](https://cookbook.openai.com/examples/using_logprobs#1-using-logprobs-to-assess-confidence-for-classification-tasks)


In [73]:
completion = client.chat.completions.create(
  model="mlx-community/Llama-3.2-3B-Instruct-4bit",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello!"}
  ],
  # stream=True,
  logprobs=True,
  top_logprobs=2, 
)

print(completion.choices[0].message)

ChatCompletionMessage(content='Hello! How can I assist you today?', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)


In [74]:
completion

ChatCompletion(id='chatcmpl-a4a5b3e89c', choices=[Choice(finish_reason='stop', index=0, logprobs=ChoiceLogprobs(content=[ChatCompletionTokenLogprob(token='Hello', bytes=[72, 101, 108, 108, 111], logprob=-0.140625, top_logprobs=[TopLogprob(token='Hello', bytes=[72, 101, 108, 108, 111], logprob=-0.140625), TopLogprob(token='How', bytes=[72, 111, 119], logprob=-2.078125)]), ChatCompletionTokenLogprob(token='!', bytes=[33], logprob=-0.03125, top_logprobs=[TopLogprob(token='!', bytes=[33], logprob=-0.03125), TopLogprob(token='.', bytes=[46], logprob=-3.375)]), ChatCompletionTokenLogprob(token=' How', bytes=[32, 72, 111, 119], logprob=-0.71875, top_logprobs=[TopLogprob(token=' It', bytes=[32, 73, 116], logprob=-0.6875), TopLogprob(token=' How', bytes=[32, 72, 111, 119], logprob=-0.71875)]), ChatCompletionTokenLogprob(token=' can', bytes=[32, 99, 97, 110], logprob=0.0, top_logprobs=[TopLogprob(token=' can', bytes=[32, 99, 97, 110], logprob=0.0), TopLogprob(token=' may', bytes=[32, 109, 97, 12

In [80]:
response = client.chat.completions.create(
    model="mlx-community/Llama-3.2-3B-Instruct-4bit",
    messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello!"}
    ],
    temperature=0,
    stream=True,
    logprobs=True,
    top_logprobs=2, 
)

for chunk in response:
    print(chunk)
    print(chunk.choices[0].delta.content)
    print("****************")

ChatCompletionChunk(id='chatcmpl-16b6ac7760', choices=[Choice(delta=ChoiceDelta(content='', function_call=None, refusal=None, role='assistant', tool_calls=None), finish_reason=None, index=0, logprobs=ChoiceLogprobs(content=None, refusal=None, token='Hello', logprob=-0.140625, bytes=[72, 101, 108, 108, 111], top_logprobs=[{'token': 'Hello', 'logprob': -0.140625, 'bytes': [72, 101, 108, 108, 111]}, {'token': 'How', 'logprob': -2.078125, 'bytes': [72, 111, 119]}]))], created=1732545699, model='mlx-community/Llama-3.2-3B-Instruct-4bit', object='chat.completion.chunk', service_tier=None, system_fingerprint=None, usage=None)

****************
ChatCompletionChunk(id='chatcmpl-16b6ac7760', choices=[Choice(delta=ChoiceDelta(content='', function_call=None, refusal=None, role='assistant', tool_calls=None), finish_reason=None, index=0, logprobs=ChoiceLogprobs(content=None, refusal=None, token='!', logprob=-0.03125, bytes=[33], top_logprobs=[{'token': '!', 'logprob': -0.03125, 'bytes': [33]}, {'tok